In [1]:
from data_loader.dataloader import *
from data_loader import dataloader
import numpy as np
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
from model.model import *
from tqdm import tqdm

In [2]:
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

In [3]:
model = torch.load('trained_models/1e-05_61.pt')
model.cuda()

ood_model(
  (classifier): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [4]:
metadata = './data/HAM10000_metadata.csv'
images = './data/'
batch=1024

In [5]:
_,_,test_data = dataloader.prepare_data(metadata,all_classes[:-1],images,create_split=True,split=(0.64,0.16,0.2),batch=batch)

In [6]:
outputs = []
labels = []
total = 0
correct = 0
with torch.no_grad():
    for data,target in tqdm(test_data): 
        data,target = data.cuda(),target.cuda()
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        outputs.append(predicted.detach().cpu())
        labels.append(target.detach().cpu())
#         smax = F.softmax(output,1)
#         print(predicted)
#         total += target.size(0)
#         correct += (predicted == target).sum().item()
# print(correct/total)

100%|██████████| 2/2 [05:54<00:00, 177.17s/it]


In [7]:
pred=np.hstack(outputs)
label=np.hstack(labels)

In [8]:
import sklearn.metrics as sk
reprt=sk.classification_report(label,pred, target_names=all_classes[:-1])

/home/abinav/anaconda3/envs/cancer/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
print(reprt)

              precision    recall  f1-score   support

          nv       0.75      0.81      0.78       355
         mel       0.83      0.61      0.70       580
         bkl       0.66      0.88      0.76       632
         bcc       0.96      0.54      0.69       291
       akiec       0.00      0.00      0.00         0
        vasc       0.94      0.90      0.92        81
          df       0.72      0.75      0.74        65

    accuracy                           0.74      2004
   macro avg       0.70      0.64      0.66      2004
weighted avg       0.78      0.74      0.74      2004



In [ ]:
model = torch.load('trained_models/1e-05_61.pt')
model.cuda()

In [ ]:
import sklearn.metrics as sk
def classification_metrics(prediction,labels):
    
    accuracy = sk.accuracy_score(labels,prediction)
    f1_score = sk.f1_score(labels,prediction)
    precision = sk.precision_score(labels,prediction)
    recall = sk.recall_score(labels,prediction)
    return accuracy, f1_score,precision, recall

In [ ]:
a,f,p,r = classification_metrics(outputs,labels)

In [ ]:
outputs